# Pricing Derivatives with Binomial Trees

This notebook will function to show off the speed advantages of derivatives pricing in kdb/q, as well as  my own binomial tree algorithms. I will cover European options, American options, Barrier options (up and out, up and in, down and out, down and in), as well as Lookback options(floating and fixed). 
The code for the algorithms may be found in this depository, except for my path dependent options.

In [1]:
\cd C:\q\w32\Lattice
\l LatticePricing.q
\c 100 100

#1 European Options

In [2]:
//Lets get started with our first function binTree
d:0.85 //downstep
u:1.2 //upstep
t:5 //time steps(initial is 1)
deltaT:0.5 //timesteps per year
r:0.01 //risk  free interest rate
y:0.07 //yield
s:100 //initial stock price
k:110 //strike price
binTree[d;u;t;deltaT;r;y;s;k]

time upsteps downsteps stockPrice call     put     
---------------------------------------------------
1    0       0         100        15.00194 14.6282 
2    1       0         120        25.61291 6.672286
2    0       1         85         4.541366 22.73077
3    2       0         144        42.35433 1.475174
3    1       1         102        9.128259 11.93629
3    0       2         72.25      0        33.75312
4    3       0         172.8      66.78524 0       
4    2       1         122.4      18.34803 2.965137
4    1       2         86.7       0        21.0271 
4    0       3         61.4125    0        46.81751
5    4       0         207.36     97.36    0       
5    3       1         146.88     36.88    0       
5    2       2         104.04     0        5.96    
5    1       3         73.695     0        36.305  
5    0       4         52.20062   0        57.79938


binTree creates the entire tree in table form for a European option, returning the values of calls and puts each timestep. binTree is flexible enough to handle Cox-Ross-Rubinstein trees as well as Jarrow-Rudd trees, forcing q = .5 when u =! 1/d 

binTree is also very fast

In [3]:
d:0.99 
u:1%d 
deltaT:.01
t:10
\t binTree[d;u;t;deltaT;r;y;s;k]
t:500
\t binTree[d;u;t;deltaT;r;y;s;k]
t:1000
\t binTree[d;u;t;deltaT;r;y;s;k]
t:2000
\t binTree[d;u;t;deltaT;r;y;s;k]
//time in milliseconds

0


101


452


3266


binTreeAmerican calculates the value of exercising the option at any time step and replaces the value of the call/put if the exercise value plus interest is higher than holding out

In [4]:
d:0.85
u:1.2 
t:5 
deltaT:0.5
r:0.07
y:0.01
s:100 
k:110 
binTreeAmerican[d;u;t;deltaT;r;y;s;k]

time upsteps downsteps stockPrice call     put      exCall   exPut   
---------------------------------------------------------------------
1    0       0         100        13.3309  16.76962 0        11.50274
2    1       0         120        23.461   6.966135 11.10711 0       
2    0       1         85         4.150496 27.76777 0        27.76777
3    2       0         144        39.99667 1.389267 36.46528 0       
3    1       1         102        8.596671 13.03927 0        8.580065
3    0       2         72.25      0        40.48718 0        40.48718
4    3       0         172.8      65.03692 0        65.03692 0       
4    2       1         122.4      17.80576 2.877504 12.84168 0       
4    1       2         86.7       0        24.12994 0        24.12994
4    0       3         61.4125    0        50.31817 0        50.31817
5    4       0         207.36     97.36    0        97.36    0       
5    3       1         146.88     36.88    0        36.88    0       
5    2       2      

Barrier functions barrierUpOutEur, barrierUpInEur, barrierDownOutEur, and barrierDownInEur calculate European barrier options by taking a dictionary containing downstep, upstep, timesteps(including first), timesteps per year, risk free rate, yield, stock, strike, and barrier
these functions take a dictionary instead of 9 parameters due to kdb/q's valence limits

In [5]:
//Down and In
dict:(`d`u`t`deltaT`r`y`s`k`b)!(0.940856 1.062862 5 0.25 .01 0.06 10 8 9 )
//strike is 8,  barrier is 9
barrierDownInEur[dict]

time upsteps downsteps stockPrice call      put       
------------------------------------------------------
1    0       0         10         0.2109056 0.01015001
2    1       0         10.62862   0.1057168 0         
2    0       1         9.40856    0.3171503 0.02035084
3    2       0         11.29676   0         0         
3    1       1         10         0.2119628 0         
3    0       2         8.8521     0.4239255 0.04080355
4    3       0         12.00689   0         0         
4    2       1         10.62862   0         0         
4    1       2         9.408561   0.4249867 0         
4    0       3         8.328552   0.4249867 0.08181138
5    4       0         12.76167   0         0         
5    3       1         11.29676   0         0         
5    2       2         10         0         0         
5    1       3         8.852101   0.8521009 0         
5    0       4         7.835968   0         0.1640323 


In [6]:
//Down and Out
dict:(`d`u`t`deltaT`r`y`s`k`b)!(0.940856 1.062862 5 0.25 .01 0.06 10 10 9 )
//changed the strike to 10
barrierDownOutEur[dict]

time upsteps downsteps stockPrice call         put
--------------------------------------------------
1    0       0         10         0.4918511    0  
2    1       0         10.62862   0.8252806    0  
2    0       1         9.40856    0.1608839    0  
3    2       0         11.29676   1.332119     0  
3    1       1         10         0.3225733    0  
3    0       2         8.8521     0            0  
4    3       0         12.00689   2.024147     0  
4    2       1         10.62862   0.6467606    0  
4    1       2         9.408561   8.96476e-007 0  
4    0       3         8.328552   0            0  
5    4       0         12.76167   2.76167      0  
5    3       1         11.29676   1.296757     0  
5    2       2         10         1.79744e-006 0  
5    1       3         8.852101   0            0  
5    0       4         7.835968   0            0  


In [7]:
//Up and In
dict:(`d`u`t`deltaT`r`y`s`k`b)!(0.940856 1.062862 5 0.25 .01 0.06 10 9 11 )
//changed the strike to 9, barrier to 11
barrierUpInEur[dict]

time upsteps downsteps stockPrice call      put
-----------------------------------------------
1    0       0         10         0.8012411 0  
2    1       0         10.62862   1.321544  0  
2    0       1         9.40856    0.2849495 0  
3    2       0         11.29676   2.078378  0  
3    1       1         10         0.5713256 0  
3    0       2         8.8521     0         0  
4    3       0         12.00689   3.02165   0  
4    2       1         10.62862   1.145511  0  
4    1       2         9.408561   0         0  
4    0       3         8.328552   0         0  
5    4       0         12.76167   3.76167   0  
5    3       1         11.29676   2.296757  0  
5    2       2         10         0         0  
5    1       3         8.852101   0         0  
5    0       4         7.835968   0         0  


In [8]:
//Up and out
barrierUpOutEur[dict]

time upsteps downsteps stockPrice call      put       
------------------------------------------------------
1    0       0         10         0.3093911 0.108635  
2    1       0         10.62862   0.2481325 0.01834925
2    0       1         9.40856    0.3721987 0.1994646 
3    2       0         11.29676   0         0         
3    1       1         10         0.4975071 0.03679036
3    0       2         8.8521     0.2487536 0.3631374 
4    3       0         12.00689   0         0         
4    2       1         10.62862   0.4987525 0         
4    1       2         9.408561   0.4987525 0.0737649 
4    0       3         8.328552   0         0.6543278 
5    4       0         12.76167   0         0         
5    3       1         11.29676   0         0         
5    2       2         10         1.000002  0         
5    1       3         8.852101   0         0.1478991 
5    0       4         7.835968   0         1.164032  


In [9]:
//These are also fast
dict:(`d`u`t`deltaT`r`y`s`k`b)!(0.940856 1.062862 500 0.01 .01 0.06 10 8 9 )
//changed timesteps to 500, deltaT to 0.01

\t barrierDownInEur[dict]
\t barrierDownOutEur[dict]
\t barrierUpInEur[dict]
\t barrierUpOutEur[dict]

184


856


186


958


Note that the algorithm does not produce alternative values for each path at each node, but works backward to arrive at the correct price. 

# Path Dependent Options

The first path dependent options are lookback options, both floating and fixed. Note, the algorithm is only optimized for Cox-Ross-Rubinstein trees and do not work with a tilt, so we must make d=1/u .


In [10]:
//floating
d:0.85
u:1%d
t:6
deltaT:0.5
r:0.01
y:0.05
s:100
floatingLBCallCRR[d;u;t;deltaT;r;y;s] //call
floatingLBPutCRR[d;u;t;deltaT;r;y;s] //put

15.38564


27.66744


In [11]:
//fixed
d:0.85
u:1%d
t:20
deltaT:0.5
r:0.01
y:0.05
s:100
k:110
fixedLBCallCRR[d;u;t;deltaT;r;y;s;k]
fixedLBPutCRR[d;u;t;deltaT;r;y;s;k]

28.15555


62.30451


KDB does not have a "BigInteger" data type, so this implementation is limited by overflow and precision 

In [12]:
d:0.85
u:1%d
deltaT:0.25
r:0.01
y:0.05
s:100
k:110

t:10
\t fixedLBPutCRR[d;u;t;deltaT;r;y;s;k]
fixedLBPutCRR[d;u;t;deltaT;r;y;s;k]

t:100
\t fixedLBPutCRR[d;u;t;deltaT;r;y;s;k]
fixedLBPutCRR[d;u;t;deltaT;r;y;s;k]

\t fixedLBPutCRR[d;u;t;deltaT;r;y;s;k]
t:1000
\t fixedLBPutCRR[d;u;t;deltaT;r;y;s;k]
fixedLBPutCRR[d;u;t;deltaT;r;y;s;k]

0


41.81428


8


95.93182


5


1449


109.9996


A path dependent binary tree with 1000 time steps gives 2^1000, or 1.071509e+301 paths, but this algo will find the lookback value  in 1.194 seconds


Let us observe the convergence when we set our option duration to 6 months, increasing the number of time steps as we decrease deltaT and u,d accordingly

In [48]:
t:50
vol:0.3
d:exp(neg vol*sqrt(.5%(t-1)))
u:1%d
deltaT:.5%(t-1)
r:0.07
y:0.04
s:100
k:100
fixedLBCallCRR[d;u;t;deltaT;r;y;s;k]

17.29649


In [40]:
t:100
vol:0.3
d:exp(neg vol*sqrt(.5%(t-1)))
u:1%d
deltaT:.5%(t-1)
r:0.07
y:0.04
s:100
k:100
fixedLBCallCRR[d;u;t;deltaT;r;y;s;k]

17.77524


In [41]:
t:500
vol:0.3
d:exp(neg vol*sqrt(.5%(t-1)))
u:1%d
deltaT:.5%(t-1)
r:0.07
y:0.04
s:100
k:100
fixedLBCallCRR[d;u;t;deltaT;r;y;s;k]

18.43463


In [71]:
t:1000
vol:0.3
d:exp(neg vol*sqrt(.5%(t-1)))
u:1%d
deltaT:.5%(t-1)
r:0.07
y:0.04
s:100
k:100
fixedLBCallCRR[d;u;t;deltaT;r;y;s;k]
fixedLBCallCRR[d;u;t;deltaT;r;y;s;k] * exp(.5*neg r) //discount the payoff back 6 months

18.59483


17.95527


We can compare this solution to a Monte Carlo Solution

In [72]:
\cd C:\q\w32\qtips-master
\l stat.q
\l deriv.q

n:20000
c:1b //call 
S:100
k:100
s:.3
r:.07

t:(til 126)%251
.deriv.mcstat raze .deriv.mc[S;s;r;t;.deriv.lb[c;k]] peach 20#n

ev | 18.27656
err| 0.04964209
n  | 400000


We can see that the binomial tree is converging towards the monte carlo solution